<a href="https://colab.research.google.com/github/marceloosg/ibge/blob/master/Trustly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trustly Test Case

# New Section

## Data Exploration/Description

In [1]:
import os

In [2]:
#fn=[f for f in os.listdir(".")  if 'csv' in f][0]
fn='dataset_test_ds_v2 - Atualizado.csv'

In [3]:
import pandas as pd

In [4]:
df=pd.read_csv(fn, encoding= 'latin1')

In [5]:
df.describe()

,SAFRA,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,CEP,TARGET
count,11169.000000,10437.000000,10942.000000,11169.000000,11169.000000,10263.000000,11169.000000,11008.000000,10821.000000,11057.000000,11169.000000,1.116900e+04,11169.000000
mean,201906.522339,0.106448,19.750658,531.046901,1396.048438,0.186982,0.177903,4345.434375,0.397468,0.008592,0.030531,2.006559e+07,0.010744
std,3.447787,0.308425,25.442371,906.626021,1736.590512,0.640979,0.382448,11527.310213,0.489397,0.092297,0.172051,1.019638e+07,0.103100
min,201901.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.050301e+06,0.000000
25%,201904.000000,0.000000,2.800000,37.520000,30.000000,0.000000,0.000000,77.865000,0.000000,0.000000,0.000000,8.412006e+06,0.000000
50%,201907.000000,0.000000,10.000000,135.000000,1321.000000,0.000000,0.000000,415.185000,0.000000,0.000000,0.000000,2.132109e+07,0.000000
75%,201910.000000,0.000000,25.300000,520.000000,1988.000000,0.000000,0.000000,2804.085000,1.000000,0.000000,0.000000,2.918217e+07,0.000000
max,201912.000000,1.000000,100.000000,8540.000000,15616.000000,11.000000,1.000000,143268.550000,1.000000,1.000000,1.000000,3.808006e+07,1.000000


In [6]:
df.head()

,SAFRA,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,CEP,TARGET
0,201901,NaN,8.1,9.99,1968,0.0,0,15.15,0.0,0.0,0,SP,São Paulo,8412006,0
1,201910,0.0,4.4,35.00,1369,0.0,0,63.98,1.0,0.0,0,RJ,Rio de Janeiro,23580304,0
2,201906,0.0,0.7,52.99,1228,0.0,0,98.84,0.0,0.0,0,MG,Belo Horizonte,30421310,0
3,201910,0.0,63.3,810.00,0,0.0,1,9237.21,0.0,0.0,0,SP,São Paulo,8253410,0
4,201902,0.0,4.1,17.50,0,0.0,1,27.70,1.0,0.0,0,ES,Vitória,29017186,0


Observaçöes:
- SAFRA -> data-mes
- V1 -> presença de Nans
- V11 -> UF 
- V12 -> cidade CEP 
- Target -> variável binária

In [7]:
df.isna().mean()

SAFRA     0.000000
V1        0.065539
V2        0.020324
V3        0.000000
V4        0.000000
V5        0.081117
V6        0.000000
V7        0.014415
V8        0.031158
V9        0.010028
V10       0.000000
V11       0.000000
V12       0.000000
CEP       0.000000
TARGET    0.000000
dtype: float64

Aqui temos algumas opçoes: 
- imputar os valores faltantes
- adicionar colunas indicando o valor nulo.
- utilizar algoritmos que lidam com os valores nulos


In [8]:
df.TARGET.value_counts()

0    11049
1      120
Name: TARGET, dtype: int64

In [9]:
df.SAFRA.min(), df.SAFRA.max()

(201901, 201912)

In [10]:
df.groupby("V11").TARGET.mean()

V11
ES    0.006663
MG    0.013579
RJ    0.014939
SP    0.009024
Name: TARGET, dtype: float64

Transformar ceps en geodata

In [11]:
df.CEP.isna().any()

False

In [12]:
ceps=df.CEP.unique()

In [16]:
from urllib.request import urlopen
import json
import pandas as pd

class geo_ceps:
  base="http://viacep.com.br/ws/CEP/json/"

  def __init__(self, ceps):
    self.ceps = [f"{cep:08}" for cep in ceps]
    self.i=0
    self.cep_list = []
    self.cep_ibge = None
    self.br_ceps=pd.read_csv('br.csv', sep = ',')
    self.get_ceps_from_url()    

  def cep_lat_lon(self, cep):
    response = urlopen(self.base.replace("CEP",str(cep)))
    json_data = response.read().decode('utf-8', 'replace')
    d = json.loads(json_data)
    self.i=self.i+1
    if 'erro' in list(d.keys()):
      d={'cep':cep, 'ibge': -1}
    self.cep_list.append(d)
  
  def get_ceps_from_url(self):
    _ = [self.cep_lat_lon(cep) for cep in self.ceps]

  def merge_ibge_geodata_with_cep(self):
    cep_ibge=pd.DataFrame(self.cep_list, index=range(len(self.cep_list))).loc[:,['cep', 'ibge']]
    cep_ibge["ibge"] = cep_ibge.ibge.astype(int)
    geo_cep_ibge=cep_ibge.merge(self.br_ceps.loc[:,["CD_GEOCODM", "LAT", "LONG"]],left_on = 'ibge', right_on = 'CD_GEOCODM', how='left').loc[:,["cep", "LAT", "LONG"]]
    geo_cep_ibge=geo_cep_ibge.groupby("cep").mean().reset_index()
    geo_cep_ibge["icep"]=geo_cep_ibge.cep.apply(lambda s: s.replace("-","")).astype(int)
    return geo_cep_ibge
class transform_geo_cep:
  def __init__(self, df):
    self.df = df
    ceps=df.CEP.unique()
    gc=geo_ceps(ceps)
    self.geo_cep_ibge=gc.merge_ibge_geodata_with_cep()        

  def get_df(self, cols = ['SAFRA', 'V1', 'V2', 'V3', 'V4','V4','V6', 'V7','V8', 'V9', 'V10', 'LAT', 'LONG', 'TARGET']):    
    return self.df.merge(self.geo_cep_ibge.loc[:,["LAT","LONG", "icep" ]] , left_on= 'CEP' , right_on = 'icep').loc[:, cols]    


In [18]:
df_geo = transform_geo_cep(df)

In [19]:
dgeo=df_geo.get_df()

In [29]:
df_geo.geo_cep_ibge

,cep,LAT,LONG,icep
0,07050-301,-23.461082,-46.469757,7050301
1,07095-080,-23.461082,-46.469757,7095080
2,07132-220,-23.461082,-46.469757,7132220
3,07176-191,-23.461082,-46.469757,7176191
4,07263-290,-23.461082,-46.469757,7263290
...,...,...,...,...
95,38051-066,-19.735032,-47.830288,38051066
96,38055-480,-19.735032,-47.830288,38055480
97,38060-030,-19.735032,-47.830288,38060030
98,38080-025,-19.735032,-47.830288,38080025


In [21]:
dgeo.isna().mean()

SAFRA     0.000000
V1        0.065539
V2        0.020324
V3        0.000000
V4        0.000000
V4        0.000000
V6        0.000000
V7        0.014415
V8        0.031158
V9        0.010028
V10       0.000000
LAT       0.013430
LONG      0.013430
TARGET    0.000000
dtype: float64

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
splitter = GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 7)
split = splitter.split(dgeo, groups=dgeo['SAFRA'])
train_inds, test_inds = next(split)
train = dgeo.iloc[train_inds]
test = dgeo.iloc[test_inds]
len(test)

In [23]:
dgeo.groupby(['LAT','LONG']).nunique()

,,SAFRA,V1,V2,V3,V4,V4,V6,V7,V8,V9,V10,TARGET
LAT,LONG,,,,,,,,,,,,
-23.813449,-46.689383,12,2,700,1470,1182,1182,2,3090,2,2,2,2
-23.780555,-46.542984,12,2,150,214,190,190,2,261,2,2,2,2
-23.499323,-47.457853,12,2,193,233,218,218,2,299,2,1,2,2
-23.461082,-46.469757,12,2,213,310,307,307,2,426,2,2,2,2
-22.886346,-47.087354,12,2,206,277,265,265,2,381,2,2,2,2
-22.876652,-43.227875,12,2,435,654,624,624,2,1158,2,2,2,2
-22.837660,-43.030613,12,2,181,206,207,207,2,271,2,2,2,2
-22.764556,-43.399625,12,2,108,126,112,112,2,149,2,2,2,2
-22.759835,-43.450345,12,2,177,231,207,207,2,298,2,2,2,2


,cep,LAT,LONG
0,07095-080,-23.461082,-46.469757
1,08253-410,-23.813449,-46.689383
2,08412-006,-23.813449,-46.689383
3,08420-400,-23.813449,-46.689383
4,21060-640,-22.876652,-43.227875
5,23580-304,-22.876652,-43.227875
6,29017-186,-20.283943,-40.308770
7,29090-790,-20.283943,-40.308770
8,30421-310,-19.928504,-43.947771
9,36080-330,-21.801644,-43.462184


In [ ]:
df.head()

,SAFRA,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,CEP,TARGET
0,201901,NaN,8.1,9.99,1968,0.0,0,15.15,0.0,0.0,0,SP,São Paulo,8412006,0
1,201910,0.0,4.4,35.00,1369,0.0,0,63.98,1.0,0.0,0,RJ,Rio de Janeiro,23580304,0
2,201906,0.0,0.7,52.99,1228,0.0,0,98.84,0.0,0.0,0,MG,Belo Horizonte,30421310,0
3,201910,0.0,63.3,810.00,0,0.0,1,9237.21,0.0,0.0,0,SP,São Paulo,8253410,0
4,201902,0.0,4.1,17.50,0,0.0,1,27.70,1.0,0.0,0,ES,Vitória,29017186,0


,SAFRA,V1,V2,V3,V4,V4,V6,V7,V8,V9,V10,LAT,LONG,TARGET
0,201901,NaN,8.1,9.99,1968,1968,0,15.15,0.0,0.0,0,-23.813449,-46.689383,0
1,201905,0.0,100.0,105.00,0,0,1,1331.67,NaN,0.0,0,-23.813449,-46.689383,0
2,201906,NaN,30.6,19.46,1757,1757,0,51.69,0.0,0.0,0,-23.813449,-46.689383,0
3,201912,0.0,0.1,1.99,1421,1421,0,1.91,1.0,0.0,1,-23.813449,-46.689383,0
4,201908,0.0,7.3,50.95,2247,2247,0,105.57,1.0,0.0,0,-23.813449,-46.689383,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11164,201911,0.0,4.9,24.00,723,723,0,41.03,1.0,0.0,0,-22.837660,-43.030613,0
11165,201912,0.0,16.7,1000.00,1963,1963,0,4453.65,0.0,0.0,0,-22.837660,-43.030613,0
11166,201907,0.0,2.6,24.00,1997,1997,0,39.45,1.0,0.0,0,-22.837660,-43.030613,0
11167,201909,1.0,10.0,594.99,0,0,1,2113.93,0.0,0.0,0,-22.837660,-43.030613,0
